# All credits to this notebook: https://www.kaggle.com/code/renatoreggiani/optv-lightgbm

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from itertools import combinations
import warnings
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from warnings import simplefilter
import joblib

warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

is_train = False
is_infer = True
N_Folds = 4

train = pd.read_csv('../input/optiver-trading-at-the-close/train.csv')
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].std() + train.groupby('stock_id')['ask_size'].std()
train = train.dropna(subset=['target'])

def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['bid_ask_volume_diff'] = df['ask_size'] - df['bid_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price']) / 2
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0)
        
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        'date_id', 
    ], inplace=True)
        
    gc.collect()
    
    return df

y = train['target'].values
X = feat_eng(train.drop(columns='target'))

y_min = np.min(y)
y_max = np.max(y)

params = {
    'learning_rate': 0.018,
    'max_depth': 9,
    'n_estimators': 600,
    'num_leaves': 440,
    'objective': 'mae',
    'random_state': 42,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01
}


kf = KFold(n_splits=N_Folds, shuffle=True, random_state=42)
mae_scores = []

if is_train:
    for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)

        m = lgb.train(params, train_data, valid_sets=[train_data, valid_data], verbose_eval=50, early_stopping_rounds=50)
        print(f"Fold {fold+1} Trainning finished.")

        model_filename = f"model_fold_{fold+1}.pkl"
        joblib.dump(m, model_filename)
        y_pred_valid = m.predict(X_valid)

        y_pred_valid = np.nan_to_num(y_pred_valid)
        y_valid = np.nan_to_num(y_valid)
        mae = mean_absolute_error(y_valid, y_pred_valid)
        mae_scores.append(mae)

    # 计算五折平均的MAE
    average_mae = np.mean(mae_scores)
    print(f"5 fold MAE: {average_mae}")

def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    
    return out


if is_infer:
    import optiver2023
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    predictions = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        feat = feat_eng(test)
        fold_prediction = 0
        for fold in range(0, N_Folds):
            model_filename = f"/kaggle/input/lgb-models-optv2/model_fold_{fold+1}.pkl"
            m = joblib.load(model_filename)
            fold_prediction += m.predict(feat)   
        
        fold_prediction /= N_Folds
        fold_prediction = zero_sum(fold_prediction, test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
        clipped_predictions = np.clip(fold_prediction, y_min, y_max)
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
